In [ ]:
exp_name = 'exp207'
sub_fn = f'{exp_name}.csv'
model_path = f"/kaggle/input/lmsys-download-{exp_name}/models/models/{exp_name}/"

!python inference_classifier.py --model_name "/kaggle/input/gemma-2/transformers/gemma-2-9b-it-4bit/1/gemma-2-9b-it-4bit" \
                        --model_path $model_path \
                        --max_length 2200 \
                        --batch_size 4
                        
!mv submission.csv $sub_fn

In [ ]:
!python inference_generator.py
!mv submission.csv "exp143.csv"

In [ ]:
!python task_inference.py

In [ ]:
import pandas as pd
from scipy.special import softmax

submission = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/sample_submission.csv')
submission = submission[['id']].copy()
task_labels = pd.read_csv('test_df_with_task.csv', usecols=['id', 'preds'])
submission = pd.merge(submission, task_labels, on='id', how='left')

target_cols = ['winner_model_a', 'winner_model_b', 'winner_tie']
submission[target_cols] = 0

ensemble_weights = {
    0: {
        'exp143_winner_model_a': 0.3738846949105062,
        'exp207_winner_model_a': 0.8185591053090249,
        'exp143_winner_model_b': 0.4209922412906948,
        'exp207_winner_model_b': 0.6141531183490367,
        'exp143_winner_tie': 0.5276501496129475,
        'exp207_winner_tie': 0.7768422538548456
    },
    1: {
        'exp143_winner_model_a': 0.4307237083489731,
        'exp207_winner_model_a': 0.6566998454132951,
        'exp143_winner_model_b': 0.45171711765958505,
        'exp207_winner_model_b': 0.547718778533672,
        'exp143_winner_tie': 0.40003259635775384,
        'exp207_winner_tie': 0.5619871248678524
    },
    2: {
        'exp143_winner_model_a': 0.3687531440776909,
        'exp207_winner_model_a': 0.8451276458778113,
        'exp143_winner_model_b': 0.3943420050410785,
        'exp207_winner_model_b': 0.65189469638862,
        'exp143_winner_tie': 0.40371112027790873,
        'exp207_winner_tie': 0.9689076736750359
    },
    3: {
        'exp143_winner_model_a': 0.5524999275313329,
        'exp207_winner_model_a': 0.7791045270995678,
        'exp143_winner_model_b': 0.41519018987020406,
        'exp207_winner_model_b': 0.4568914030363267,
        'exp143_winner_tie': 0.6397793902096839,
        'exp207_winner_tie': 0.7284585096953455
    },
    -1: {
        'exp143_winner_model_a': 0.4674247584365413,
        'exp207_winner_model_a': 0.7589456105830781,
        'exp143_winner_model_b': 0.47908478842377866,
        'exp207_winner_model_b': 0.5604379012141305,
        'exp143_winner_tie': 0.527185127482252,
        'exp207_winner_tie': 0.7193006354491285
    }
}

dfs = []
for label in submission.preds.unique():
    sub = submission[submission['preds'] == label].copy()

    columns_mapping = {col: f'preds_{col}' for col in target_cols}
    for model_name in ['exp207', 'exp143']:
        df = pd.read_csv(f'{model_name}.csv')
        df.rename(columns=columns_mapping, inplace=True)
        
        weights = ensemble_weights.get(label, ensemble_weights[-1])
        sub = pd.merge(sub, df, on='id', how='left')
        for col in target_cols:
            sub[col] += sub['preds_'+col] * weights[f'{model_name}_{col}'] 
        sub.drop(['preds_'+col for col in target_cols], axis=1, inplace=True)
    dfs.append(sub)
    
submission = pd.concat(dfs)
submission[target_cols] = softmax(submission[target_cols].values, axis=1)
submission[['id', 'winner_model_a', 'winner_model_b', 'winner_tie']].to_csv('submission.csv', index=False)
submission

In [ ]:
task_labels

In [ ]:
df